In [12]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, RadiusNeighborsClassifier, RadiusNeighborsRegressor
import json
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity
import statistics
import math
import operator
from tqdm import tqdm

# ОЧЕНЬ ДОЛГО СЧИТАЕТСЯ! уходим в склерн

In [10]:
def filter_similar(sorted_similarity):
    filtered_sorted_similarity_list = []
    first_similarity = sorted_similarity[0][1]
    if first_similarity < 0.7:
            return None
    for similar_el in sorted_similarity[:5]:
        similarity_value = similar_el[1]
        if similarity_value - first_similarity > 0.15 or similarity_value < 0.5:
            break
        filtered_sorted_similarity_list.append(similar_el)
    return filtered_sorted_similarity_list

In [4]:
with open("./text_maps/text_0.json", "r") as f:
        potential_recommended_text_map = json.load(f)

In [5]:
word_dp = pd.read_csv("word_db.csv", sep = ",", header=None)
word_dp.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,0.025718,-0.004776,0.031789,0.018760,-0.015405,-0.078441,-0.002449,-0.008062,-0.001146,0.010007,...,0.017842,-0.041569,0.077341,0.012496,0.049400,-0.037332,0.001658,0.010653,-0.036526,-0.000000
1,0.018746,0.003426,0.020213,0.056091,-0.006876,-0.030144,0.021508,-0.038640,-0.008778,0.014135,...,0.000866,-0.069830,0.107995,0.041295,0.074002,-0.027547,-0.023377,-0.026141,-0.028869,-0.062423
2,0.030954,0.012660,-0.013010,0.058922,-0.015895,-0.004156,0.043479,-0.097470,0.013334,-0.000223,...,-0.012979,-0.054481,0.129445,0.047809,0.064443,-0.009311,-0.068905,-0.063734,-0.017192,-0.050362
3,0.029982,0.046834,-0.016939,0.034865,-0.004499,-0.025411,0.057243,-0.126531,0.015994,-0.012146,...,-0.063004,0.012827,0.101894,0.074595,0.044913,-0.022268,-0.048629,-0.074751,0.018179,-0.035168
4,0.023059,0.052791,-0.031716,0.051026,-0.040494,-0.007667,0.052596,-0.087467,-0.004326,0.018472,...,-0.052820,0.033877,0.062752,0.100702,0.030436,0.018350,-0.031551,-0.050780,0.061213,-0.059490


In [6]:
trigramms =[]
with open ("trigramm_db.txt", "r", encoding = "utf-8") as f:
    for trig in f.readlines(): 
        trigramms.append(trig[:-1])

In [13]:
def trigramms_recommendation (rec_text_map, marked_trigramms_db, trigrmms_list):
    sentences_map = rec_text_map['sentences_map']
    calc_text_dict = {}
    for sentence_ind in tqdm(range(len(sentences_map))):
        for word in sentences_map[sentence_ind]['sentence_words']:
            text_word_vec = np.array(word['lex_vector']).reshape(1, -1)
            text_word_dist_dict = {}
            for word_db_ind in range(len(marked_trigramms_db)):
                marked_trigramms_db_word_vec = marked_trigramms_db.iloc[word_db_ind][:-1].values.reshape(1, -1)
                similarity = cosine_similarity(text_word_vec,marked_trigramms_db_word_vec)
                text_word_dist_dict[word_db_ind] = similarity
           
            sorted_x = sorted(text_word_dist_dict.items(), key=operator.itemgetter(1), reverse = True)
            filtered_sorted_list = filter_similar(sorted_x)
            if not filtered_sorted_list:
                calc_text_dict[word['lex_trigram']] = None
            else:
                similar_trigrams= []
                for similar_el in filtered_sorted_list:
                    similar_trigrams.append((similar_el, trigrmms_list[similar_el[0]],list(marked_trigramms_db.iloc[similar_el[0]])[-1] ))
                calc_text_dict[word['lex_trigram']] = similar_trigrams
                #=====ЗДЕСЬ ПР ПОДГРУЗКЕ НАДО БУДЕТ РАЗМЕТИТЬ ЦЕЛЕВУЮ ПЕРЕМЕННУЮВ СООТВТЕТСВИИ С КОРРЕКТНОСТЬЮ ОТВЕТА И ДАЛЕЕ ПЛЯСАТЬ
                
    vocab_recommendation_dict = {}
    for key in calc_text_dict.keys():
        print(key , "===>", calc_text_dict[key],'\n')
        understdanding_prob_list = []
        if (calc_text_dict[key]):
            for trgrm in calc_text_dict[key]:
                understdanding_prob = trgrm[2]
                understdanding_prob_list.append(understdanding_prob)
            understdanding_prob_mean = statistics.mean(understdanding_prob_list) 
            vocab_recommendation_dict[key] = understdanding_prob
    positive_understanding_prob = 0
    whole_understading_prob = 0 
    for key in vocab_recommendation_dict.keys():
        print(key , "===>", vocab_recommendation_dict[key],'\n')
        if (vocab_recommendation_dict[key] > 0):
            positive_understanding_prob += vocab_recommendation_dict[key]
            whole_understading_prob += abs(vocab_recommendation_dict[key])
        elif(vocab_recommendation_dict[key] < 0):
            whole_understading_prob += abs(vocab_recommendation_dict[key])
    understanding_prob = positive_understanding_prob/whole_understading_prob
    print("understanding_prob",understanding_prob)
        
trigramms_recommendation(potential_recommended_text_map,word_dp,  trigramms)


100%|██████████| 5/5 [01:02<00:00, 11.13s/it]

[ Указом президента ===> [((667, array([[0.80777536]])), 'администрацией президента президента', 0.0838445869857746), ((668, array([[0.79732806]])), 'президента и правительством', 0.0), ((669, array([[0.74179425]])), 'и правительством ]', 0.07080278907515195), ((474, array([[0.73092272]])), 'пресс-службу президента Украины', -0.045863594791982494), ((666, array([[0.68948096]])), 'доработан администрацией президента', 0.07677773082452256)] 

Указом президента России ===> [((331, array([[0.96677111]])), '[ Президенты России', -0.02347102930266891), ((474, array([[0.91138107]])), 'пресс-службу президента Украины', -0.045863594791982494), ((667, array([[0.87828387]])), 'администрацией президента президента', 0.0838445869857746), ((668, array([[0.85525247]])), 'президента и правительством', 0.0), ((332, array([[0.84500587]])), 'Президенты России Владимир', -0.028437229511918544)] 

президента России Бориса ===> [((332, array([[0.95053364]])), 'Президенты России Владимир', -0.028437229511918

In [2]:
def extract_text_map_features(text_map_path):
    with open(text_map_path, "r") as f:
        rec_text_map = json.load(f)
        #load text features
        rec_text_features_vector = []
        rec_text_features_vector.append(rec_text_map['lix'])
        rec_text_features_vector.append(rec_text_map['ttr'])
        rec_text_features_vector.extend(rec_text_map['sent_properties'])

        #load sentence features
        sentence_map = rec_text_map['sentences_map']
        recommended_sentences = []
        for sentence_ind in range(len(sentence_map)):
            rec_sent_feat = []
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['negation'])
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['coreference'])
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['vozvr_verb'])
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['prich'])
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['deepr'])
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['case_complexity'])
            rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['mean_depend_length'])
            rec_sent_vec = np.array(rec_sent_feat).reshape(1, -1)
            recommended_sentences.append(rec_sent_vec)

        #load words features
        recommended_words = []
        for sentence in sentence_map:
            for word_element in sentence['sentence_words']:
                word_vector = word_element['lex_vector']
                word_vector = np.array(word_vector).reshape(1, -1)
                recommended_words.append(word_vector)
    return recommended_words, recommended_sentences, rec_text_features_vector
                
wrd_feat, snts_feat, text_feat = extract_text_map_features ("./text_maps/text_0.json")

In [ ]:
def predict_text_understanding(rec_text_words_vectors,rec_text_sentences_vectors, rec_text_txt_vector, word_competence_db, sent_competence_db, text_competence_db):
    words_predictions = []
    for word in rec_text_words_vectors:
        word_arr = np.array(word).reshape(1, -1)
        ######
        prediction = word_model.predict(word_arr)
        word_predictions.append(prediction)
    positive_understanding = 0 
    negative_understanding = 0 
    for w_pred in word_predictions:
        if w_pred[0] > 0:
            positive_understanding += 1
        elif w_pred[0] < 0:
            negative_understanding += 1    
    words_understanding_probability = positive_understanding / (positive_understanding + negative_understanding)
    #print(words_understanding_probability)
    
    sent_predictions = []
    for sent in rec_text_sentences_vectors:
        sent_arr = np.array(sent).reshape(1, -1)
        prediction = sent_model.predict(sent_arr)
        sent_predictions.append(prediction)
    positive_understanding = 0 
    negative_understanding = 0 
    for snt_pred in sent_predictions:
        if snt_pred[0] == 1:
            positive_understanding += 1
        elif snt_pred[0] == 0:
            negative_understanding += 1    
    sent_understanding_probability = positive_understanding / (positive_understanding + negative_understanding)
    #print(sent_understanding_probability)
    
    text_arr = np.array(rec_text_txt_vector).reshape(1, -1)
    text_understanding_pobability = text_model.predict(text_arr)[0]
    #print(text_understanding_pobability)
    
    return words_understanding_probability, sent_understanding_probability, text_understanding_pobability
    
predict_text_understanding(wrd_feat, snts_feat, text_feat, neigh_words, neigh_sent, neigh_text)    


In [ ]:
%%time
prediction = []
def calculate_accuracy(database, debug = False):
    db_arr = np.array([np.array(list(database.iloc[ind])) for ind in range(len(database))])
    db_arr_X = db_arr[:,:-1]
    db_arr_y = db_arr[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(db_arr_X, db_arr_y, test_size=0.15)
    for test_vector_ind in tqdm(range(len(X_test))):
        test_vector_arr = np.array(X_test[test_vector_ind]).reshape(1,-1)
        test_vector_dict = {}
        for train_vector_ind in range(len(X_train)):
            train_vector = X_train[train_vector_ind]
            train_vector_arr = np.array(train_vector).reshape(1,-1)
            similarity = cosine_similarity(test_vector_arr, train_vector_arr)
            test_vector_dict[train_vector_ind] = similarity
        sorted_test_vector_dict = sorted(test_vector_dict.items(), key=operator.itemgetter(1), reverse = True)
        if debug: print(sorted_test_vector_dict[:5])
        sorted_test_vector_dict = filter_similar(sorted_test_vector_dict)
        if sorted_test_vector_dict:
            #normalization
            sum_similarity = 0
            for sort_element in sorted_test_vector_dict[:5]:
                sum_similarity += sort_element[1][0][0]

            normalized_similarity_list = []
            for sort_element in sorted_test_vector_dict[:5]:
                norm_sim = sort_element[1][0][0] / sum_similarity
                target_variable = y_train[sort_element[0]]
                normalized_similarity_list.append((norm_sim, target_variable))
            if debug: print(normalized_similarity_list)

            math_expectation = 0
            for norm_el in normalized_similarity_list:
                math_expectation += norm_el[0] * norm_el[1] 
            prediction.append(math_expectation)
            if debug:
                print(math_expectation)
                print()
        else:
            prediction.append(-0)
    if debug: print(prediction)
    print(r2_score(y_test,prediction))
    return y_test, prediction
true, prde = calculate_accuracy(word_db, debug = False)

In [ ]:
def predict_text_understanding(rec_text_words_vectors,rec_text_sentences_vectors, rec_text_txt_vector, word_model, sent_model, text_model):
    words_predictions = []
    for word in rec_text_words_vectors:
        word_arr = np.array(word).reshape(1, -1)
        prediction = word_model.predict(word_arr)
        word_predictions.append(prediction)
    positive_understanding = 0 
    negative_understanding = 0 
    for w_pred in word_predictions:
        if w_pred[0] > 0:
            positive_understanding += 1
        elif w_pred[0] < 0:
            negative_understanding += 1    
    words_understanding_probability = positive_understanding / (positive_understanding + negative_understanding)
    #print(words_understanding_probability)
    
    sent_predictions = []
    for sent in rec_text_sentences_vectors:
        sent_arr = np.array(sent).reshape(1, -1)
        prediction = sent_model.predict(sent_arr)
        sent_predictions.append(prediction)
    positive_understanding = 0 
    negative_understanding = 0 
    for snt_pred in sent_predictions:
        if snt_pred[0] == 1:
            positive_understanding += 1
        elif snt_pred[0] == 0:
            negative_understanding += 1    
    sent_understanding_probability = positive_understanding / (positive_understanding + negative_understanding)
    #print(sent_understanding_probability)
    
    text_arr = np.array(rec_text_txt_vector).reshape(1, -1)
    text_understanding_pobability = text_model.predict(text_arr)[0]
    #print(text_understanding_pobability)
    
    return words_understanding_probability, sent_understanding_probability, text_understanding_pobability
    
predict_text_understanding(wrd_feat, snts_feat, text_feat, neigh_words, neigh_sent, neigh_text)    


In [ ]:
%%time
prediction = []
def calculate_accuracy(database, debug = False):
    db_arr = np.array([np.array(list(database.iloc[ind])) for ind in range(len(database))])
    db_arr_X = db_arr[:,:-1]
    db_arr_y = db_arr[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(db_arr_X, db_arr_y, test_size=0.15)
    for test_vector_ind in tqdm(range(len(X_test))):
        test_vector_arr = np.array(X_test[test_vector_ind]).reshape(1,-1)
        test_vector_dict = {}
        for train_vector_ind in range(len(X_train)):
            train_vector = X_train[train_vector_ind]
            train_vector_arr = np.array(train_vector).reshape(1,-1)
            similarity = cosine_similarity(test_vector_arr, train_vector_arr)
            test_vector_dict[train_vector_ind] = similarity
        sorted_test_vector_dict = sorted(test_vector_dict.items(), key=operator.itemgetter(1), reverse = True)
        if debug: print(sorted_test_vector_dict[:5])
        sorted_test_vector_dict = filter_similar(sorted_test_vector_dict)
        if sorted_test_vector_dict:
            #normalization
            sum_similarity = 0
            for sort_element in sorted_test_vector_dict[:5]:
                sum_similarity += sort_element[1][0][0]

            normalized_similarity_list = []
            for sort_element in sorted_test_vector_dict[:5]:
                norm_sim = sort_element[1][0][0] / sum_similarity
                target_variable = y_train[sort_element[0]]
                normalized_similarity_list.append((norm_sim, target_variable))
            if debug: print(normalized_similarity_list)

            math_expectation = 0
            for norm_el in normalized_similarity_list:
                math_expectation += norm_el[0] * norm_el[1] 
            prediction.append(math_expectation)
            if debug:
                print(math_expectation)
                print()
        else:
            prediction.append(-0)
    if debug: print(prediction)
    print(r2_score(y_test,prediction))
    return y_test, prediction
true, prde = calculate_accuracy(word_db, debug = False)